In [1]:
# Spark imports.
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import (StructField,StringType, 
                               IntegerType,StructType)

In [2]:
# Start the Spark session.
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
# Read from a path relative to the one where the Jupyter notebook is.
# Can also read from large distributed files like from HDFS, this may
# involve large datasets in Amazon EC2.
df = spark.read.json('Data/people.json')

In [4]:
# Missing data will be replaced with `null`.
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
# Check the schema of the dataframe. Some type inferences may not be
# proper, like assuming an Integer has Long type.
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
# Check the column names. Its not a function but an attribute so
# it can be called without ().
df.columns

['age', 'name']

In [7]:
# Count of elements in the dataframe.
df.count()

3

In [8]:
# Statistical summary of the dataframe.
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [9]:
# Some data types make it easier to infer schema (like tabular formats
# such as csv). Often we have to set the schema ourselves if we are
# dealing with a .read method that doesn't have inferSchema() built-in.
data_schema = [
    # Name, type, and whether it can be null.
    StructField('age', IntegerType(), True),
    StructField('name', StringType(), True)
]

In [10]:
final_struct = StructType(fields=data_schema)

In [11]:
# Read the JSON but using the schema we defined.
df = spark.read.json('Data/people.json', schema=final_struct)

In [12]:
# Now age can be a integer instead of a long. Also by enforcing
# a schema we can guarantee constraints.
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [13]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [14]:
# How do we actually grab data from our dataframe? This only returns a
# column object, we can validate by checking the retrunt
type(df['age'])

pyspark.sql.column.Column

In [15]:
# This will instead create a new data frame only with the column we want.
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [16]:
# The type is a whole dataframe, not a column object.
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [17]:
# This returns a list of row objects, so we access the first one.
type(df.head(2)[0])

pyspark.sql.types.Row

In [18]:
# You can also select multiple columns!
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [19]:
# You can add a new column, for each row it will take the value of the 'age'
# column object and multiply by two. This doesn't alter the original object.
df.withColumn('new_age', df['age'] * 2).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     60|
|  19| Justin|     38|
+----+-------+-------+



In [20]:
# This is a command to just rename columns and nothing else.
df.withColumnRenamed('age', 'new_age').show()

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



In [21]:
# A UDF (User defined function) will be a function that returns a value then
# converted into a Spark type (e.g. StringType) to be used in operations.
# For example if you want to create a new column, you can't just put the
# values in there, it has to contain a Column object, which this will return.
concat_udf = f.udf(lambda age, name: "{}.{}".format(age, name), StringType())

In [22]:
# If you want to add a new column but also drop an old one. Use the UDF to
# concatenate age and name into a value that will be put into the new column.
df.withColumn('info', concat_udf(df.age, df.name)).drop('age')

DataFrame[name: string, info: string]

In [23]:
# Now, lets do some SQL operations! But you can't just write the name of the
# python variable of a dataframe, so lets "temporarily" alias it to a table
# name to be able to SQL-query it.
df.createOrReplaceTempView('people')

In [24]:
# SQL query from a literal string.
results = spark.sql('''
   SELECT *
   FROM people
   WHERE age >= 25 AND age <= 35
''')

In [25]:
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

